# Resnet with CIFAR 10

In [20]:
import torch
import torch.nn as nn

import torch.optim as optim

import torchvision
import torchvision.transforms as transforms

In [21]:
device = 'cuda' if torch.cuda.is_available() else 'cpu'

torch.manual_seed(777)
if device =='cuda':
    torch.cuda.manual_seed_all(777)
device

'cuda'

## mean 과 normalize 계산하는 방법

In [22]:
transform = transforms.Compose([
    transforms.ToTensor()
])

trainset = torchvision.datasets.CIFAR10(root='./cifar10', train=True, download=True, transform=transform)

print(trainset.data.shape)
# 각 축에 대한 평균과 std 계산
train_data_mean = trainset.data.mean( axis=(0,1,2) )
train_data_std = trainset.data.std( axis=(0,1,2) )


print(train_data_mean)
print(train_data_std)

train_data_mean = train_data_mean / 255
train_data_std = train_data_std / 255

print(train_data_mean)
print(train_data_std)

Files already downloaded and verified
(50000, 32, 32, 3)
[125.30691805 122.95039414 113.86538318]
[62.99321928 62.08870764 66.70489964]
[0.49139968 0.48215841 0.44653091]
[0.24703223 0.24348513 0.26158784]


In [23]:
transform_train = transforms.Compose([
                transforms.RandomCrop(32,padding=4),
                transforms.ToTensor(),
                transforms.Normalize(train_data_mean, train_data_std)
])

transform_test = transforms.Compose([
                transforms.ToTensor(),
                transforms.Normalize(train_data_mean, train_data_std)

])

trainset = torchvision.datasets.CIFAR10(root='./cifar10',
                                        train=True,
                                        transform=transform_train,
                                        download=True)
trainloader = torch.utils.data.DataLoader(dataset=trainset,
                                          batch_size=256,
                                          shuffle=True)
testset = torchvision.datasets.CIFAR10(root='./cifar10',
                                       train=False,
                                       transform = transform_test,
                                       download=True)
testloader = torch.utils.data.DataLoader(dataset=testset,
                                          batch_size=256,
                                          shuffle=False)
print(len(trainloader))
classes = ('plane', 'car', 'bird', 'cat',
           'deer', 'dog', 'frog', 'horse', 'ship', 'truck')

Files already downloaded and verified
Files already downloaded and verified
196


In [24]:
import torchvision.models.resnet as resnet

In [25]:
conv1x1 = resnet.conv1x1
Bottleneck = resnet.Bottleneck
BasicBlock = resnet.BasicBlock

In [26]:
class ResNet(nn.Module):

    def __init__(self, block, layers, num_classes=1000, zero_init_residual=False):
        super(ResNet, self).__init__()
        self.inplanes = 16
        self.conv1 = nn.Conv2d(3, 16, kernel_size=3, stride=1, padding=1,
                               bias=False)
        self.bn1 = nn.BatchNorm2d(16)
        self.relu = nn.ReLU(inplace=True)
        #self.maxpool = nn.MaxPool2d(kernel_size=3, stride=2, padding=1)
        
        self.layer1 = self._make_layer(block, 16, layers[0], stride=1)
        self.layer2 = self._make_layer(block, 32, layers[1], stride=1)
        self.layer3 = self._make_layer(block, 64, layers[2], stride=2)
        self.layer4 = self._make_layer(block, 128, layers[3], stride=2)
        
        self.avgpool = nn.AdaptiveAvgPool2d((1, 1))
        self.fc = nn.Linear(128 * block.expansion, num_classes)

        for m in self.modules():
            if isinstance(m, nn.Conv2d):
                nn.init.kaiming_normal_(m.weight, mode='fan_out', nonlinearity='relu')
            elif isinstance(m, nn.BatchNorm2d):
                nn.init.constant_(m.weight, 1)
                nn.init.constant_(m.bias, 0)

        # Zero-initialize the last BN in each residual branch,
        # so that the residual branch starts with zeros, and each residual block behaves like an identity.
        # This improves the model by 0.2~0.3% according to https://arxiv.org/abs/1706.02677
        if zero_init_residual:
            for m in self.modules():
                if isinstance(m, Bottleneck):
                    nn.init.constant_(m.bn3.weight, 0)
                elif isinstance(m, BasicBlock):
                    nn.init.constant_(m.bn2.weight, 0)

    # [3,4,6,3]
    # make_layer(block, 16, layers[0], stride=1)
    def _make_layer(self, block, planes, blocks, stride=1):
        downsample = None
        if stride != 1 or self.inplanes != planes * block.expansion:
            downsample = nn.Sequential(
                conv1x1(self.inplanes, planes * block.expansion, stride),
                nn.BatchNorm2d(planes * block.expansion),
            )

        layers = []
        layers.append(block(self.inplanes, planes, stride, downsample))
        self.inplanes = planes * block.expansion
        for _ in range(1, blocks):
            layers.append(block(self.inplanes, planes))

        return nn.Sequential(*layers)

    def forward(self, x):
        x = self.conv1(x)
        #x.shape =[1, 16, 32,32]
        x = self.bn1(x)
        x = self.relu(x)
        #x = self.maxpool(x)

        x = self.layer1(x)
        #x.shape =[1, 128, 32,32]
        x = self.layer2(x)
        #x.shape =[1, 256, 32,32]
        x = self.layer3(x)
        #x.shape =[1, 512, 16,16]
        x = self.layer4(x)
        #x.shape =[1, 1024, 8,8]
        
        x = self.avgpool(x)
        x = x.view(x.size(0), -1)
        x = self.fc(x)

        return x

In [27]:
resnet50 = ResNet(resnet.Bottleneck, [3,4,6,3], 10, True).to(device)
#1(conv1) + 9(layer1) + 12(layer2) + 18(layer3) + 9(layer4) +1(fc)= ResNet50
resnet50

ResNet(
  (conv1): Conv2d(3, 16, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1), bias=False)
  (bn1): BatchNorm2d(16, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
  (relu): ReLU(inplace=True)
  (layer1): Sequential(
    (0): Bottleneck(
      (conv1): Conv2d(16, 16, kernel_size=(1, 1), stride=(1, 1), bias=False)
      (bn1): BatchNorm2d(16, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
      (conv2): Conv2d(16, 16, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1), bias=False)
      (bn2): BatchNorm2d(16, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
      (conv3): Conv2d(16, 64, kernel_size=(1, 1), stride=(1, 1), bias=False)
      (bn3): BatchNorm2d(64, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
      (relu): ReLU(inplace=True)
      (downsample): Sequential(
        (0): Conv2d(16, 64, kernel_size=(1, 1), stride=(1, 1), bias=False)
        (1): BatchNorm2d(64, eps=1e-05, momentum=0.1, affine=True, track_ru

In [28]:
a = torch.Tensor(1,3,32,32).to(device)
out = resnet50(a)
print(out)

tensor([[nan, nan, nan, nan, nan, nan, nan, nan, nan, nan]], device='cuda:0',
       grad_fn=<AddmmBackward>)


In [29]:
criterion = nn.CrossEntropyLoss().to(device)
optimizer = optim.SGD(resnet50.parameters(),lr = 0.1, momentum=0.9,weight_decay=5e-4)
lr_sche = optim.lr_scheduler.StepLR(optimizer,step_size=10,gamma=0.5)

## acc check

In [33]:
def acc_check(net, test_set, epoch, save=1):
    correct = 0
    total = 0
    with torch.no_grad():
        for data in test_set:
            images, labels = data
            images = images.to(device)
            labels = labels.to(device)
            outputs = net(images)

            _, predicted = torch.max(outputs.data, 1)

            total += labels.size(0)
            correct += (predicted == labels).sum().item()
    
    acc = (100 * correct / total)
    print('Accuracy of the network on the 10000 test images: %d %%' % acc)
    if save:
        torch.save(net.state_dict(), "./model/model_epoch_{}_acc_{}.pth".format(epoch, int(acc)))
    return acc

In [34]:
print(len(trainloader))
epochs = 150

for epoch in range(epochs):

    running_loss = 0.0
    lr_sche.step()
    for i, data in enumerate(trainloader, 0):
        inputs, labels = data
        inputs = inputs.to(device)
        labels = labels.to(device)

        optimizer.zero_grad()
        hypo = resnet50(inputs)
        cost = criterion(hypo,labels)
        cost.backward()
        optimizer.step()
        running_loss += cost.item()
        if i % 30 == 29:
            print('[%d, %5d] loss: %.3f' %
                  (epoch + 1, i + 1, running_loss / 30))
            running_loss = 0.0
    acc = acc_check(resnet50, testloader, epoch, save=0)
print('Finished Training')

196
[1,    30] loss: 0.549
[1,    60] loss: 0.541
[1,    90] loss: 0.516
[1,   120] loss: 0.505
[1,   150] loss: 0.474
[1,   180] loss: 0.501
Accuracy of the network on the 10000 test images: 77 %
[2,    30] loss: 0.458
[2,    60] loss: 0.472
[2,    90] loss: 0.471
[2,   120] loss: 0.482
[2,   150] loss: 0.466
[2,   180] loss: 0.488
Accuracy of the network on the 10000 test images: 80 %
[3,    30] loss: 0.433
[3,    60] loss: 0.432
[3,    90] loss: 0.455
[3,   120] loss: 0.435
[3,   150] loss: 0.447
[3,   180] loss: 0.439
Accuracy of the network on the 10000 test images: 79 %
[4,    30] loss: 0.411
[4,    60] loss: 0.424
[4,    90] loss: 0.449
[4,   120] loss: 0.421
[4,   150] loss: 0.440
[4,   180] loss: 0.452
Accuracy of the network on the 10000 test images: 80 %
[5,    30] loss: 0.408
[5,    60] loss: 0.385
[5,    90] loss: 0.394
[5,   120] loss: 0.395
[5,   150] loss: 0.421
[5,   180] loss: 0.419
Accuracy of the network on the 10000 test images: 80 %
[6,    30] loss: 0.378
[6,    6

KeyboardInterrupt: ignored

In [35]:
correct = 0
total = 0

with torch.no_grad():
    for data in testloader:
        images, labels = data
        images = images.to(device)
        labels = labels.to(device)
        outputs = resnet50(images)
        
        _, predicted = torch.max(outputs.data, 1)
        
        total += labels.size(0)
        
        correct += (predicted == labels).sum().item()

print('Accuracy of the network on the 10000 test images: %d %%' % (
    100 * correct / total))

Accuracy of the network on the 10000 test images: 80 %
